In [10]:
import os
import glob
import itertools
from PIL import Image
from tqdm.notebook import tqdm
from concurrent.futures import ProcessPoolExecutor

In [2]:
raw_images = []
for file in glob.glob("../dataset/raw_images/*.png"):
    raw_images.append(file)
print(f"Number of raw images: {len(raw_images)}")

Number of raw images: 534


In [3]:
crop_size = 600

In [4]:
caransarea_path = f"../dataset/carvansara_{crop_size}x{crop_size}/carvansara/"
other_path = f"../dataset/carvansara_{crop_size}x{crop_size}/other/"

for path in [caransarea_path, other_path]:
    if not os.path.exists(path):
        os.makedirs(path)

In [8]:
def process_and_save_image(image_path, crop_size):
    # find image name
    image_name = os.path.basename(image_path)[:-4]
    # find label
    if "g" in image_name or "p" in image_name:
        output_path = other_path
    else:
        output_path = caransarea_path
    # read image
    image = Image.open(image_path);
    # crop google 
    x = 1920//2
    y = 2160//4
    w = 800
    h = 700
    google = image.crop((x-w//2, 3*y-h//2, x + w//2, 3*y + h//2 + 100))
    google = google.resize((crop_size,crop_size))
    google.save(os.path.join(output_path, f"{image_name}_google_{crop_size}x{crop_size}.png"))
    # crop bing
    x = 1920//2
    y = 2160//4
    w = 800
    h = 700
    bing = image.crop((x-w//2, y-h//2, x + w//2, y + h//2 + 100))
    bing = bing.resize((crop_size,crop_size))
    bing.save(os.path.join(output_path, f"{image_name}_bing_{crop_size}x{crop_size}.png"))

In [12]:
with ProcessPoolExecutor() as executor:
    results = list(tqdm(executor.map(process_and_save_image,
                                     raw_images,
                                     itertools.repeat(crop_size)),
                                     total=len(raw_images)))